In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
sys.path.insert(0, '../../')
from seqtables import seqtables
from seqtables.utils.insilica_sequences import generate_sequence
from seqtables.io import create_scratch_data

In [3]:
import xarray as xr
import numpy as np

In [4]:
vals = np.array([0,1,2,3,4,5,6,7,8]).reshape(3,3)
vals2 = np.array([0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15]).reshape(2,8)
x1 = xr.DataArray(vals, dims=('x', 'y'), coords={'x': [0, 1, 2], 'y': [3,4,5]})
x2 = xr.DataArray(vals2, dims=('x', 'y'), coords={'x': [2,3], 'y': [1,2,3,4,5,6,7,8]})
x1a = xr.Dataset({'t1': x1})
x2a = xr.Dataset({'t2': x2})
xr.merge([x1a, x2a])


<xarray.Dataset>
Dimensions:  (x: 4, y: 8)
Coordinates:
  * y        (y) int64 1 2 3 4 5 6 7 8
  * x        (x) int64 0 1 2 3
Data variables:
    t1       (x, y) float64 nan nan 0.0 1.0 2.0 nan nan nan nan nan 3.0 4.0 ...
    t2       (x, y) float64 nan nan nan nan nan nan nan nan nan nan nan nan ...

In [5]:
from seqtables.internals import constructor_ops

In [6]:
from seqtables.internals import sam_to_arr
import numpy as np

In [7]:
from seqtables.utils import sam_tools
sample_sam_file = './fake_algn_data.sam'
df = sam_tools.read_sam(sample_sam_file, chunks=None)
dft = pd.concat([d for d in df])
df = sam_tools.read_sam(sample_sam_file, chunks=None)
df3 = pd.concat([d for d in df])
dft = dft.iloc[:5000]
df3 = df3.iloc[-5000:]
df3['pos'] += 100

In [8]:
from seqtables.xarray_mods import st_merge

In [9]:
df3['header'] = df3['header'].apply(lambda x: x + '_2')

In [10]:
t1 = seqtables.seq_tables.df_to_dataset(dft, 'NT', dft['header'], ref_name='t1', ref_to_pos_dict={'t1': 't1_position'})
t2 = seqtables.seq_tables.df_to_dataset(df3, 'NT', df3['header'], ref_name='t1', ref_to_pos_dict={'t1': 't1_position'})

-1
-2
36
-1
-2
36


In [11]:
xr.merge([t1,t2])['t1.sequence_table'].shape

(10000, 400)

In [20]:
st_merge.merge([t1, t2])['t1.sequence_table']

AttributeError: 'numpy.ndarray' object has no attribute '_data'

In [16]:
t1.variables.values

<bound method Frozen.values of Frozen(OrderedDict([('t1_position', <xarray.IndexVariable 't1_position' (t1_position: 300)>
array([  1,   2,   3, ..., 298, 299, 300])), ('t1.read', <xarray.IndexVariable 't1.read' (t1.read: 5000)>
array(['insilica_seq_1', 'insilica_seq_3', 'insilica_seq_2', ...,
       'insilica_seq_4998', 'insilica_seq_4999', 'insilica_seq_5000'],
      dtype='|S17')), ('t1.insertion_table_pos_data', <xarray.IndexVariable 't1.insertion_table_pos_data' (t1.insertion_table_pos_data: 1924)>
array([('insilica_seq_4', 88, 1), ('insilica_seq_9', 226, 1),
       ('insilica_seq_19', 215, 1), ..., ('insilica_seq_4987', 15, 4),
       ('insilica_seq_4987', 15, 5), ('insilica_seq_4999', 47, 1)], dtype=object)), ('insertion_data', <xarray.IndexVariable 'insertion_data' (insertion_data: 2)>
array(['letter', 'quality'],
      dtype='|S7')), ('t1.sequence_table', <xarray.Variable (t1.read: 5000, t1_position: 300)>
array([['C', 'C', 'G', ..., 'T', 'G', 'T'],
       ['C', 'C', 'G', ...,

In [ ]:
t1.variables['t1.sequence_table']

In [ ]:
getattr(t1.variables['t1.sequence_table'], 'no_conflicts')(t2.variables['t1.sequence_table'])

In [ ]:
arr = np.array([(1,2.,'Hello'),(2,3.,"World")], 
               dtype=[('foo', 'i4'),('bar', 'f4'), ('baz', 'a10')])
recordarr = arr.view(dtype=np.dtype((np.record, arr.dtype)), 
                      type=np.recarray)
recordarr.foo

In [ ]:
arr

In [ ]:
'ltr_code'

In [ ]:
x = np.zeros(3, dtype={'col1':('i1',10,'title 1'), 'col2':('f4',3,'title 2')})
x

In [ ]:
t3['t1.sequence_table'].values

In [ ]:
t3['t1.sequence_table'] = t3['t1.sequence_table'].astype('S1')

In [ ]:
t2['t1.quality_table']

In [ ]:
xr.merge([t1,t2])

In [ ]:
t1['t1.sequence_table'].values.astype('S1').shape

In [ ]:
t1.attrs['seqtable']['references']['t1']['dimension_names']['insertion_positions_multiindex']

In [ ]:
# current naive solution, merge data arrays but have each dimension be unique
vals1 = np.random.normal(size=(6,2))
vals2 = np.random.normal(size=(3,3))
x1 = xr.Dataset(
    {'table1': (['x', 'y'], vals1)},
    coords={
        'x': np.arange(6),
        'y': np.arange(2)
    }
)
x2 = xr.Dataset(
    {'table2': (['x', 'y'], vals2)},
    coords={
        'x': np.arange(10, 10+3),
        'y': np.arange(8, 8+3)
    }
)

index=pd.MultiIndex.from_tuples([(0, 0, 1), (1, 1, 1), (2, 2, 1)], names=('x3', 'y3', 'z3'))
vals3 = np.random.normal(size=(3,3))
x3 = xr.Dataset(
        {'table3-multiindex': (['multi-index', 'cols'], vals3)},        
        coords={'multi-index': index}
)

xr.merge([x1, x2, x3], join='exact')['table1']

In [ ]:
vals3

In [ ]:
index

In [ ]:
test = pd.DataFrame([['A', 'B'], ['C', 'E'], ['F', 'G']], index=pd.MultiIndex.from_tuples([(0, 'a', 1), (1, 'b', 1), (2, 'c', 1)]))

In [ ]:
test

In [ ]:
test.to_xarray()[0]

In [ ]:
t2['t2.sequence_table']

In [ ]:
import xarray as xr


In [ ]:
t3 = xr.merge([t1, t2], compat='no_conflicts')

In [ ]:
t3['t2.sequence_table']

In [ ]:
t3['t1.sequence_table'] = t3['t1.sequence_table'][:, :100]

In [ ]:
t3['t1.sequence_table'].shape

In [ ]:
t3['t2.sequence_table'].nbytes

In [ ]:
t2['t2.sequence_table'].nbytes

In [ ]:
t3['t2.sequence_table']

In [ ]:
t1['t1.sequence_table'].nbytes

In [ ]:
t3 = xr.concat([t1, t2], dim='read')
t3

In [ ]:
t3['t1.sequence_table']

In [ ]:
t2['t1.sequence_table']

In [ ]:
t1['t1.sequence_table']

In [ ]:
t1['t1.sequence_table']

In [ ]:
import xarray as xr
# xr.concat([t1,t2])

In [ ]:
t2

In [ ]:
t1

In [ ]:
tmp1, tmp2 = _

In [ ]:
tmp1.shape

In [ ]:
tmp2

In [ ]:
dft.iloc[0]['cigar']

In [ ]:
sam_to_arr.test_cpy('116M1D67M1D1I')

In [ ]:
dft['pos'].dtype

In [ ]:
np.array(dft['seq'].values, dtype='S').view('S1').reshape(dft.shape[0], -1).dtype

In [ ]:
dft.iloc[0]

In [ ]:
import time
t1= time.time()
dummy = sam_to_arr.seq_to_algn_str(
    dft['seq'].values,
    dft['qual'].values,
    dft['pos'].values,
    dft['cigar'].values, 
    np.array(list(dft.index)),
)
t2 = time.time()
(t2-t1)*100

In [ ]:
dummy[0]

In [ ]:
pd.DataFrame(dummy[3], index=pd.MultiIndex.from_tuples(dummy[2], names=('read', 'position', 'insertion')))

In [ ]:
ttt = dft.iloc[:3]
dummy = sam_to_arr.seq_to_algn_str(
    ttt['seq'].values,
    ttt['qual'].values,
    ttt['pos'].values,
    ttt['cigar'].values,
    np.array(list(ttt.index)),
    min_pos = 1
)

In [ ]:
pd.DataFrame(dummy[2], dummy[3])

In [ ]:
t1 = _[0]
t2 = _[1]

In [ ]:
t3 = pd.DataFrame((t1[:, 2]).astype('S').view('S1').reshape(t1.shape[0], -1).view(np.uint8)).astype(np.float)

In [ ]:
t3[t3==0] = np.nan

In [ ]:
t3_ins = t3.stack().astype(np.uint8)

In [ ]:
t2t = pd.DataFrame(t2)
t2t = t2t.rename(columns={p: p + 1 for p in t2t.columns}).stack().astype(np.int)
t2t

In [ ]:
t3_ins